## CSCI E-118 Introduction to Blockchain and Bitcoin

### Introduction to Solidity


#### Solidity Version Management

The [py-solc-x](https://solcx.readthedocs.io/en/latest/index.html) package that we installed provides a python api to the solidity compiler, [solc](https://docs.soliditylang.org/en/v0.8.0/installing-solidity.html).

Each version of solidity needs the corresponding compiler version.

We can see which versions we currently have.

In [5]:
import solcx

In [7]:
# here's where solidity installs compiler versions
solcx.get_solcx_install_folder()

PosixPath('/home/nodar/.solcx')

We can see which versions of solidity `solc` can compile based on our compiler versions.

In [11]:
!ls ~/.solcx

solc-v0.8.1


Or we can simply do:

In [10]:
solcx.get_installed_solc_versions()

[Version('0.8.1')]

We can see which version it defaults to:

In [8]:
solcx.get_solc_version()

Version('0.8.1')

We'll be using MINOR version 7, so we'll need to install it. Let's check the most 4 most recent installable versions:

In [13]:
 solcx.get_installable_solc_versions()[0:4]

[Version('0.8.1'), Version('0.8.0'), Version('0.7.6'), Version('0.7.5')]

We'll grap the latest version of `MINOR == 7`, which is version `0.7.6`:

In [15]:
solcx.install_solc(version='0.7.6')

Version('0.7.6')

After installing, we're still on version `0.8.1`:

In [16]:
solcx.get_solc_version()

Version('0.8.1')

But we can see that version `0.7.6` was indeed installed:

In [18]:
solcx.get_installed_solc_versions()

[Version('0.8.1'), Version('0.7.6')]

We'll switch to it, and verify that we currectly set it:

In [19]:
solcx.set_solc_version('0.7.6')
solcx.get_solc_version()

Version('0.7.6')

#### Compiling Solidity Code

In [ ]:
from solcx import set_solc_version
set_solc_version('v0.7.6')

We could write our Solidity code here, but it's better to use a text editor or and IDE.

Remix is an online IDE which has syntax highlighting, debugging, and more. We'll write our code here and just place the finished code as a simple string here.

Notice that the `pragma` is specified to use version `0.7.x` which matches what we've set our compiler version to.

In [95]:
bank_contract_source = '''
// SPDX-License-Identifier: GPL-3.0

pragma solidity >=0.7.0 <0.8.0;

contract Bank {
    
    address private owner;
    
    mapping (address => uint) private balances;
    
    event LogDeposit(address depositor, uint amount);
    
    modifier onlyOwner() {
        require(msg.sender == owner, "Caller is not owner");
        _;
    }
    
    constructor() {
        owner = msg.sender;
    }
    
    // function internal 

    function deposit() public payable returns (uint newBalance) {
        balances[msg.sender] += msg.value;
        emit LogDeposit(msg.sender, msg.value);
        return balances[msg.sender];
    }
    
    function attemptSend(address payable account, uint withdrawAmount) internal {
        bool sent = account.send(withdrawAmount);
        if ( !sent ) {
            balances[account] += withdrawAmount;
        }
    }
    
    function withdraw(uint withdrawAmount) external returns (uint newBalance) {
        if ( balances[msg.sender] >= withdrawAmount ) {
            balances[msg.sender] -= withdrawAmount;
            attemptSend(msg.sender, withdrawAmount);
        }
        
        return balances[msg.sender];
    }
    
    function balance() public view returns (uint) {
        return balances[msg.sender];
    }
    
    function balance(address account) public view onlyOwner returns (uint) {
        return balances[account];
    }
    
    fallback () external {
        revert();
    }
}
'''

Since this contract is written in the high-level Solidity language we need to compile it. We can use the `compile_source` function.

In [21]:
from solcx import compile_source

In [96]:
compiled_bank_contract = compile_source(bank_contract_source)

Under the hood, `solcx` is passing the contract source code to the solidity compiler, `solc`, through the standard input, `<stdin>`. What is returned by the compiler is the compiled EVM bytecode and also some intermediate representations and metadata. `solcx` returns these results as a nested dictionary, and is referenced through the key `<stdin>:Bank`, where `Bank` is the name of the contract we defined in the source code.

In [97]:
compiled_bank_contract.keys()

dict_keys(['<stdin>:Bank'])

If we passed in multiple contracts in the source. They would be referencable by `<stdin>:ContractName`. Accessing the sub-dictionary for our `Bank` contract, we can see a variety of information is returned.

In [98]:
compiled_bank_contract['<stdin>:Bank'].keys()

dict_keys(['abi', 'asm', 'bin', 'bin-runtime', 'devdoc', 'generated-sources', 'generated-sources-runtime', 'hashes', 'metadata', 'opcodes', 'srcmap', 'srcmap-runtime', 'storage-layout', 'userdoc', 'ast'])

For now, we only care about the `abi` and `bin`. `abi` stands for **Application Binary Interface**, it's sort of like an API. It's how external software knows how to interface with the compiled code. The compiled code itself is the `bin` (for binary). Let's inspect them.

The `abi` is human and machine readable. It tells you how to interface with the contract.

In [99]:
compiled_bank_contract['<stdin>:Bank']['abi']

[{'inputs': [], 'stateMutability': 'nonpayable', 'type': 'constructor'},
 {'anonymous': False,
  'inputs': [{'indexed': False,
    'internalType': 'address',
    'name': 'depositor',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'amount',
    'type': 'uint256'}],
  'name': 'LogDeposit',
  'type': 'event'},
 {'stateMutability': 'nonpayable', 'type': 'fallback'},
 {'inputs': [],
  'name': 'balance',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [{'internalType': 'address',
    'name': 'account',
    'type': 'address'}],
  'name': 'balance',
  'outputs': [{'internalType': 'uint256', 'name': '', 'type': 'uint256'}],
  'stateMutability': 'view',
  'type': 'function'},
 {'inputs': [],
  'name': 'deposit',
  'outputs': [{'internalType': 'uint256',
    'name': 'newBalance',
    'type': 'uint256'}],
  'stateMutability': 'payable',
  'type': 'function'},
 {'inpu

`bin` is EVM bytecode. It's readable by machines (the EVM), but not by humans. It's in hexadecimal.

In [122]:
compiled_bank_contract['<stdin>:Bank']['bin']

'608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555061053c806100606000396000f3fe6080604052600436106100435760003560e01c80632e1a7d4d14610056578063b69ef8a8146100a5578063d0e30db0146100d0578063e3d670d7146100ee57610044565b5b34801561005057600080fd5b50600080fd5b34801561006257600080fd5b5061008f6004803603602081101561007957600080fd5b8101908080359060200190929190505050610153565b6040518082815260200191505060405180910390f35b3480156100b157600080fd5b506100ba61023a565b6040518082815260200191505060405180910390f35b6100d8610281565b6040518082815260200191505060405180910390f35b3480156100fa57600080fd5b5061013d6004803603602081101561011157600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919050505061036a565b6040518082815260200191505060405180910390f35b600081600160003373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020019081526020016

Above we placed our contract into a string variable. That's a bit clumsy. We can also store it in a `.sol` file. The `.sol` extension is understood to be Solidity code.

We have a `contracts` directory, and in there a `Bank.sol` file. The contents of the file are the same as the contents of the `bank_contract` string.

We point solidity to it and compile the contract using `compile_files`. We pass in a list of files, in this case a list of size one for the single bank contract.

In [101]:
bank_contract_path = 'contracts/Bank.sol'

In [102]:
from solcx import compile_files

In [103]:
compiled_files = compile_files( [bank_contract_path] )

We get back a dictionary, as before. This time the keys are different. We didn't pass in a string, so the solidity compiler didn't get the source code from `STDIN`, it got filenames. We reference the dictionary using the key `contracts/Bank.sol:bank`. Again, if we had passed in more files they would be referencable by `ContractPath:ContractName`.

In [104]:
compiled_files.keys()

dict_keys(['contracts/Bank.sol:Bank'])

It has the same subdictionary as before.

In [105]:
compiled_files[bank_contract_path + ':' + 'Bank'].keys()

dict_keys(['abi', 'asm', 'bin', 'bin-runtime', 'devdoc', 'generated-sources', 'generated-sources-runtime', 'hashes', 'metadata', 'opcodes', 'srcmap', 'srcmap-runtime', 'storage-layout', 'userdoc', 'ast'])

We'll extract the `abi` and `bin` for later use.

In [118]:
bank_abi = compiled_files[bank_contract_path + ':' + 'Bank']['abi']
bank_bin = compiled_files[bank_contract_path + ':' + 'Bank']['bin']

#### Interfacing With the Contract

We installed [web3.py](https://web3py.readthedocs.io/en/stable/) as part of our installations.

`web3` is the software we use to interface with contracts and the Ethereum Blockchain. It's not solidity specific. It understands ABIs and EVM bytecode. We used `solc` to produce the ABI and EVM bytecode. We could have used any other language, like Vyper, to produce the ABI and EVM bytecode. So, from here on out we're agnostic about the use of Solidity. Web3 is fairly general. If we were using JavaScript instead of Python, we'd be using web3.js instead of web3.py. If a language is popular enough, someone has probably created a `web3` library in that language. We're using Python, so we'll use `web3.py`.

But what about the Blockchain? Web3 connects to the Blockchain using `Providers`. It can do so through IPC, HTTP, or WebSockets. Which one we use depends on how we plan to connect to the Blockcahin. If we were a full node, we'd have some client software, like Geth. We may also use a centralized service that hosts a full node in the cloud for us, such as Infura. For now we're not going to deploy to either the Ethereum mainnet or a testnet, so we don't need to worry about those. We'll instead use a simulated blockchain. These different options are known as backends.

We installed `EthereumTesterProvider` as part of our installations. Under the hood, it's using the [eth-tester](https://github.com/ethereum/eth-tester) library as the backend. The `EthereumTesterProvider` creates a simulated Ethereum Blockchain for us. It sets up accounts, and uses all of the main components of the real Ethereum blockchain, just without the Network portion of it.

Here's how we intialize `web3` and use the `EthreumTesterProvider` for our backend.

In [63]:
from web3 import Web3, EthereumTesterProvider

In [64]:
w3 = Web3(EthereumTesterProvider())

In [71]:
w3.clientVersion

'EthereumTester/0.5.0b3/linux/python3.6.12'

In [82]:
w3.isConnected()

True

The `web3.eth` object exposes most of the attributes and methods we'll be using. We can see that `eth-tester` has provided us with some test accounts, each of which has a an address derived from the public key.

In [85]:
w3.eth.accounts

['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
 '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF',
 '0x6813Eb9362372EEF6200f3b1dbC3f819671cBA69',
 '0x1efF47bc3a10a45D4B230B5d10E37751FE6AA718',
 '0xe1AB8145F7E55DC933d51a18c793F901A3A0b276',
 '0xE57bFE9F44b819898F47BF37E5AF72a0783e1141',
 '0xd41c057fd1c78805AAC12B0A94a405c0461A6FBb',
 '0xF1F6619B38A98d6De0800F1DefC0a6399eB6d30C',
 '0xF7Edc8FA1eCc32967F827C9043FcAe6ba73afA5c',
 '0x4CCeBa2d7D2B4fdcE4304d3e09a1fea9fbEb1528']

We'll use the first one as our default, and switch if necessary. It's not set yet, so we have to set it manually.

In [91]:
w3.eth.default_account = w3.eth.accounts[0]
w3.eth.default_account

'0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf'

From now on, if we don't manually specify an account, it will use the one above by default.

We'll instantiate a web3 contract object using our compiled bank contract.

In [119]:
bank_contract = w3.eth.contract(abi=bank_abi, bytecode=bank_bin, bytecode_runtime=bank_bin_runtime)

This object hold our contract data (bytecode) and understands the interface (ABI). It also provides us with convinient methods to interact with it in various way. Most importantly it allows us to easily deploy it to the Blockcahin.

The `Bank` contract has a constructor, but it doesn't take in any arguments. We'll reference the constructor, and then call `.transact()` on it to issue the transaction. This deploys the contract to our backend.

In [123]:
tx_hash = bank_contract.constructor().transact()

The return value of this transaction is the transaction hash. Deployment to the Blockchain, or any transaction in general, is an asynchronous process. We have to wait for our transaction to be mined, before the results are set in stone. We're using a simulated blockchain, so it's mined immediately, but we still wait for it as if we deployed to the real Blockchain, using `w3.eth.waitForTransactionReceipt`. We get back a transaction receipt. The transaction receipt contains details of the transaction like how much gas was used. Importantly it contains the address that was created when the contract was deployed.

In [125]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [126]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x5a22222c66882239df75935f7476fbf2142afa787e47af779d2ec57819f5a980'),
 'transactionIndex': 0,
 'blockNumber': 1,
 'blockHash': HexBytes('0x7aeaa284cf4a1e0a1b1485dcabe398b4955363bfa2727fb6f921a29213895e79'),
 'cumulativeGasUsed': 363960,
 'gasUsed': 363960,
 'contractAddress': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b',
 'logs': [],
 'status': 1})

We'll reinitialize our contract, setting the returned address to our contract object so that we can interface with the deployed version.

In [154]:
bank_contract = bank_contract(address=tx_receipt['contractAddress'])

In [175]:
bank_contract.address

'0xF2E246BB76DF876Cef8b38ae84130F4F55De395b'

We have the functions defined by the contract.

In [157]:
bank_contract.all_functions()

[<Function balance()>,
 <Function balance(address)>,
 <Function deposit()>,
 <Function withdraw(uint256)>]

We can issue `call` on functions that don't mutate state (and therefore don't need to be mined).

In [158]:
bank_contract.functions.balance().call()

0

We'll deposit 1 ether. We wrote our contract such that the amout deposit isn't provided by an argument to the function. Instead the amount deposited is passed in the `value` field of the transaction itself, and the contract refrences `msg.value` to extract it. We'll create a `transaction_dict` where we can specify the transaction details. The `value` field is denominated in `wei`. Since we want to send 1 ether, we'll need to convert it. We can use `w3.toWei` for that.

In [173]:
w3.toWei(1,'ether')

1000000000000000000

In [178]:
transaction_dict = {
#     'from': w3.eth.accounts[0] # default,
#     'to': bank_contract.address # default,
#     'gas': ,
#     'gasPrice': ,
    'value': w3.toWei(1, 'ether') # send this much ether as part of transaction
}

new_balance = bank_contract.functions.deposit().call( transaction_dict )

In [181]:
w3.fromWei(new_balance, 'ether')

Decimal('1')

We got back the new balance of the sender (our default account). We received a value denominated in wei and converted it to ether using `w3.fromWei`.

It shows that we got 1 ether as our new balance, which makes sense. But if we check to make sure, we'll see that it hasn't "stuck".

In [182]:
bank_contract.functions.balance().call()

0

The `Bank.balance()` function shows that there is `0` balance. What gives?

The reason is we used `.call()`. We use `.call()` which shows us the results of issuing a transaction by running the code locally. But it doesn't actually issue the transaction to the Blockchain. For that we need to use `.transact()` as we did when we deployed the contract originally.

We'll issue the transaction, using the same transaction details as before. Remember that this is going out to the Blockchain, which is an asynchronous process. We get back a transaction hash, and have to wait for the transaction receipt.

In [183]:
tx_hash = bank_contract.functions.deposit().transact( transaction_dict )
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [184]:
tx_receipt

AttributeDict({'transactionHash': HexBytes('0xd5f6b8af23a8496ba5acf1346f2e1b10da80aa59eff64535c7262c641ff2643f'),
 'transactionIndex': 0,
 'blockNumber': 2,
 'blockHash': HexBytes('0x6c131a4817b174eb19f33780baeee7a5dd7147aaa9af8fd77ace9bddc8853398'),
 'cumulativeGasUsed': 44429,
 'gasUsed': 44429,
 'contractAddress': None,
 'logs': [AttributeDict({'type': 'mined',
   'logIndex': 0,
   'transactionIndex': 0,
   'transactionHash': HexBytes('0xd5f6b8af23a8496ba5acf1346f2e1b10da80aa59eff64535c7262c641ff2643f'),
   'blockHash': HexBytes('0x6c131a4817b174eb19f33780baeee7a5dd7147aaa9af8fd77ace9bddc8853398'),
   'blockNumber': 2,
   'address': '0xF2E246BB76DF876Cef8b38ae84130F4F55De395b',
   'data': '0x0000000000000000000000007e5f4552091a69125d5dfcb7b8c2659029395bdf0000000000000000000000000000000000000000000000000de0b6b3a7640000',
   'topics': [HexBytes('0x1b851e1031ef35a238e6c67d0c7991162390df915f70eaf9098dbf0b175a6198')]})],
 'status': 1})

Let's check if it "stuck" this time.

In [185]:
bank_contract.functions.balance().call()

1000000000000000000

It did! But let's convert it to ether.

In [186]:
w3.fromWei( bank_contract.functions.balance().call(), 'ether' )

Decimal('1')

The balance of our default account is now 1 ether, as expected.